In [ ]:
# ml_model.py

import joblib
import pandas as pd
import numpy as np
import streamlit as st # Para mostrar st.error/st.info

def load_prediction_model(model_filename='tremor_prediction_model_V2.joblib'):
    """
    Carga el modelo de Machine Learning pre-entrenado.
    """
    try:
        model = joblib.load(model_filename)
        st.info(f"Modelo '{model_filename}' cargado exitosamente.")
        return model
    except FileNotFoundError:
        st.error(f"Error: El archivo del modelo '{model_filename}' no se encontró.")
        st.error("Asegúrate de que esté en la misma carpeta que este script.")
        return None
    except Exception as e:
        st.error(f"Ocurrió un error al cargar el modelo: {e}")
        return None

def make_tremor_prediction(model, df_features, expected_features_order):
    """
    Realiza una predicción utilizando el modelo cargado.
    """
    if model is None:
        st.error("No se pudo realizar la predicción: el modelo no está cargado.")
        return None, None

    # Asegurarse de que el DataFrame de entrada coincida con las características esperadas por el modelo
    # Esto es CRÍTICO para la compatibilidad del modelo
    df_for_prediction = pd.DataFrame([df_features]).reindex(columns=expected_features_order)

    # Asegurarse de que todas las columnas numéricas sean float
    for col in df_for_prediction.columns:
        if df_for_prediction[col].dtype == 'object': # Podría ser un string si hay mezclado
            try:
                df_for_prediction[col] = pd.to_numeric(df_for_prediction[col], errors='coerce')
            except:
                pass # Si no es numérico, déjalo como está (para categóricas)

    try:
        prediction = model.predict(df_for_prediction)
        probabilities = None
        if hasattr(model, 'predict_proba'):
            probabilities = model.predict_proba(df_for_prediction)
        return prediction[0], probabilities
    except Exception as e:
        st.error(f"Ocurrió un error al usar el modelo para la predicción: {e}")
        st.error("Verifica que el DataFrame de entrada (`df_features`) coincida en formato y columnas con lo que espera tu modelo entrenado.")
        return None, None